# Book Recommendations.

In [422]:
import sqlite3
from contextlib import contextmanager
from typing import List, Tuple, Dict
import numpy as np
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import normalize


class BooksRecommender:

    def __init__(self) -> None:
        
        # Model
        self.transformer = SentenceTransformer(
            "paraphrase-mpnet-base-v2" # English.
#             "paraphrase-multilingual-mpnet-base-v2" # Multilingual.
        )
        self.emb = None
        
        # Database.
        self.con = sqlite3.connect(":memory:")
        self.cur = self.con.cursor()
        self.cols = set([
            "title",
            "description",
            "url",
            "image_url",
        ])
        self.cur.execute('''
        CREATE TABLE IF NOT EXISTS books (
              id integer not null primary key asc
            , title text
            , description text
            , url text
            , image_url text
        )
        ;
        ''')
        self.con.commit()
        
    def add_books(self,
                  cols:List[str],
                  values:List[Dict[str, str]]
                 ) -> None:
        values = [[d[c] for c in cols if c in self.cols] for d in values]
        q = ",".join(["?" for c in cols if c in self.cols])
        cols = ",".join([c for c in cols if c in self.cols])
        self.cur.executemany(f'''
        INSERT INTO books ({cols}) values ({q})
        ''', values)
        self.con.commit()
        
    def update_by_id(self, _id:int, values:Dict[str, str]) -> None:
        k_v = ",".join([f"{k}=\"{v}\"" \
                        for k,v in values.items() if k in self.cols])
        self.cur.execute(f'''
        UPDATE books SET {k_v} WHERE id = ?
        ''', (_id,))
        self.con.commit()

    def get_by_title(self, title:str) -> List[Tuple[int, str, str]]:
        return [book for book in self.cur.execute('''
        SELECT * FROM books where title like ?
        ''', ("%"+title+"%",))]
    
    def get_by_id(self, _id:int) -> Tuple[int, str, str]:
        return [book for book in self.cur.execute('''
        SELECT * FROM books where id = ?
        ''', (_id,))]
    
    def apply_transformer(self) -> None:
        self.emb = normalize(
            self.transformer.encode(
            [t[0] for t in self.cur.execute(
            "select title || ' ' || description from books order by id asc;")]))
        
    def __call__(self, text:str) -> List[Tuple[str, str, str, str]]:
        query = self.transformer.encode([text]).T
        similarities = self.emb.dot(query)[:,0]
        ids = np.argpartition(similarities, len(similarities)-3)[-3:] + 1
        return [t for t in self.cur.execute(
            "select * from books where id in (?,?,?)", ids.tolist())]

In [436]:
%%time
rec = BooksRecommender()
rec.add_books(["title", "description"], [
    dict(title="Treasure Islands", description="Tax Havens and the Men Who Stole the World. This is the ugliest chapter in global economic affairs since slavery -- and secretive offshore tax havens are at the heart of the trouble. Dirty money, tax havens and the offshore system describe the ugliest and most secretive chapter in the history of global economic affairs. The World Bank has reported that the flow of dirty money across borders, out of developing countries and into rich ones, is up to ten times the amount of foreign aid that flows the other way. Most people regard tax havens as being relevant only to celebrities, crooks and spivs, and mistakenly believe that the main offshore problems are money laundering and terrorist financing. These are only small parts of the whole picture. The offshore system has been (discreetly) responsible for the greatest ever shift of wealth from poor to rich. It also undermines our democracies by offering the wealthiest members of society escape routes from tax, financial regulation and other normal democratic controls. The anti-globalisation movement tapped into an uneasiness, felt by millions, that something was rotten in the world economy. Treasure Islands brilliantly articulates the problem in a completely new way, and exposes the deep corruption that impacts on our daily lives."),
    dict(title="The book of longings.", description="Raised in a wealthy family in Sepphoris with ties to the ruler of Galilee, Ana is rebellious and ambitious, a relentless seeker with a brilliant, curious mind and a daring spirit. She yearns for a pursuit worthy of her life, but finds no outlet for her considerable talents. Defying the expectations placed on women, she engages in furtive scholarly pursuits and writes secret narratives about neglected and silenced women. When she meets the eighteen-year-old Jesus, each is drawn to and enriched by the other’s spiritual and philosophical ideas. He becomes a floodgate for her intellect, but also the awakener of her heart. Their marriage unfolds with love and conflict, humor and pathos in Nazareth, where Ana makes a home with Jesus, his brothers, James and Simon, and their mother, Mary. Here, Ana’s pent-up longings intensify amid the turbulent resistance to the Roman occupation of Israel, partially led by her charismatic adopted brother, Judas. She is sustained by her indomitable aunt Yaltha, who is searching for her long-lost daughter, as well as by other women, including her friend Tabitha, who is sold into slavery after she was raped, and Phasaelis, the shrewd wife of Herod Antipas. Ana’s impetuous streak occasionally invites danger. When one such foray forces her to flee Nazareth for her safety shortly before Jesus’s public ministry begins, she makes her way with Yaltha to Alexandria, where she eventually finds refuge and purpose in unexpected surroundings. Grounded in meticulous historical research and written with a reverential approach to Jesus’s life that focuses on his humanity, The Book of Longings is an inspiring account of one woman’s bold struggle to realize the passion and potential inside her, while living in a time, place, and culture devised to silence her."),
    dict(title="Feast of Sorrow: A Novel of Ancient Rome", description="In the twenty-sixth year of Augustus Caesar’s reign, Marcus Gavius Apicius has a singular ambition: to serve as culinary adviser to Caesar. To cement his legacy as Rome’s leading epicure, the wealthy Apicius acquires a young chef, Thrasius, for the exorbitant price of twenty thousand denarii. Apicius believes that the talented Thrasius is the key to his culinary success, and with the slave’s help he soon becomes known for his lavish parties and sumptuous meals. For his part, Thrasius finds a family among Apicius’s household, which includes his daughter, Apicata; his wife, Aelia; and her handmaiden Passia, with whom Thrasius falls passionately in love. But as Apicius draws closer to his ultimate goal, his dangerous single-mindedness threatens his young family and places his entire household at the mercy of the most powerful forces in Rome. “A gastronomical delight” (Associated Press), Feast of Sorrow is a vibrant novel, replete with love and betrayal, politics and intrigue, and sumptuous feasts that bring ancient Rome to life."),
    dict(title="The Big Short.", description="The real story of the crash began in bizarre feeder markets where the sun doesn't shine and the SEC doesn't dare, or bother, to tread: the bond and real estate derivative markets where geeks invent impenetrable securities to profit from the misery of lower- and middle-class Americans who can't pay their debts. The smart people who understood what was or might be happening were paralyzed by hope and fear; in any case, they weren't talking. Michael Lewis creates a fresh, character-driven narrative brimming with indignation and dark humor, a fitting sequel to his #1 bestseller Liar's Poker. Out of a handful of unlikely-really unlikely-heroes, Lewis fashions a story as compelling and unusual as any of his earlier bestsellers, proving yet again that he is the finest and funniest chronicler of our time."),
    dict(title="Too Big to Fail.", description="The Inside Story of How Wall Street and Washington Fought to Save the Financial System from Crisis — and Themselves. Andrew Ross Sorkin delivers the first true behind-the-scenes, moment-by-moment account of how the greatest financial crisis since the Great Depression developed into a global tsunami. From inside the corner office at Lehman Brothers to secret meetings in South Korea, and the corridors of Washington, Too Big to Fail is the definitive story of the most powerful men and women in finance and politics grappling with success and failure, ego and greed, and, ultimately, the fate of the world’s economy. “We’ve got to get some foam down on the runway!” a sleepless Timothy Geithner, the then-president of the Federal Reserve of New York, would tell Henry M. Paulson, the Treasury secretary, about the catastrophic crash the world’s financial system would experience. Through unprecedented access to the players involved, Too Big to Fail re-creates all the drama and turmoil, revealing neverdisclosed details and elucidating how decisions made on Wall Street over the past decade sowed the seeds of the debacle. This true story is not just a look at banks that were “too big to fail,” it is a real-life thriller with a cast of bold-faced names who themselves thought they were too big to fail."),
    dict(title="A Thousand Ships", description="From the Trojan women whose fates now lie in the hands of the Greeks, to the Amazon princess who fought Achilles on their behalf, to Penelope awaiting the return of Odysseus, to the three goddesses whose feud started it all, these are the stories of the women whose lives, loves, and rivalries were forever altered by this long and tragic war. A woman’s epic, powerfully imbued with new life, A Thousand Ships puts the women, girls and goddesses at the center of the Western world’s great tale ever told."),
    dict(title="The Emperor's Blades", description="The Emperor has been murdered, leaving the Annurian Empire in turmoil. Now his progeny must bury their grief and prepare to unmask a conspiracy. His son Valyn, training for the empire’s deadliest fighting force, hears the news an ocean away. He expected a challenge, but after several ‘accidents’ and a dying soldier’s warning, he realizes his life is also in danger. Yet before Valyn can take action, he must survive the mercenaries’ brutal final initiation. Meanwhile, the Emperor’s daughter, Minister Adare, hunts her father’s murderer in the capital itself. Court politics can be fatal, but she needs justice. And Kaden, heir to an empire, studies in a remote monastery. Here, the Blank God’s disciples teach their harsh ways – which Kaden must master to unlock their ancient powers. When an imperial delegation arrives, he’s learnt enough to perceive evil intent. But will this keep him alive, as long-hidden powers make their move?"),
    dict(title="The Last Kingdom", description="This is the story of the making of England in the 9th and 10th centuries, the years in which King Alfred the Great, his son and grandson defeated the Danish Vikings who had invaded and occupied three of England’s four kingdoms. The story is seen through the eyes of Uhtred, a dispossessed nobleman, who is captured as a child by the Danes and then raised by them so that, by the time the Northmen begin their assault on Wessex (Alfred’s kingdom and the last territory in English hands) Uhtred almost thinks of himself as a Dane. He certainly has no love for Alfred, whom he considers a pious weakling and no match for Viking savagery, yet when Alfred unexpectedly defeats the Danes and the Danes themselves turn on Uhtred, he is finally forced to choose sides. By now he is a young man, in love, trained to fight and ready to take his place in the dreaded shield wall. Above all, though, he wishes to recover his father’s land, the enchanting fort of Bebbanburg by the wild northern sea. This thrilling adventure—based on existing records of Bernard Cornwell’s ancestors—depicts a time when law and order were ripped violently apart by a pagan assault on Christian England, an assault that came very close to destroying England."),
    dict(title="Agincourt", description="Agincourt is one of the epic battles of history. It was fought by two badly matched armies that met in atrocious conditions on St Crispin's Day 1415, and resulted in an extraordinary victory that was celebrated in England long before Shakespeare immortalised it in Henry V. It has always been held to be the triumph of the longbow against the armoured knight, and of the common man against the feudal aristocrat, but those are history's myths. Bernard Cornwell, who has long wanted to write this story, depicts the reality behind the myths. Nicholas Hook is an English archer. He seems born to trouble and, when his lord orders him to London as part of a force sent to quell an expected Lollard uprising, Nick's headstrong behaviour leads to him being proscribed an outlaw. He finds refuge across the Channel, part of an English mercenary force protecting the town of Soissons against the French. What happened at the Siege of Soissons shocked all Europe, and propels Nick back to England where he is enrolled in the archer companyof the doughty Sir John Cornwaille, a leader of Henry V's army. The army was superb, but sickness and the unexpected French defiance at Harfleur, reduce it to near-shambolic condition. Henry stubbornly refuses to accept defeat and, in appalling weather, leads his shrunken force to what appears to be inevitable disaster. Azincourt culminates in the battle. Seen from several points of view on the English side, but also from the French ranks, the scene is vivid, convincing and compelling. Bernard Cornwell has a great understanding of men at war and battlefields and this is his masterpiece. This is what it must have been like to fight at Agincourt."),
])
rec.apply_transformer()

CPU times: user 4.08 s, sys: 563 ms, total: 4.64 s
Wall time: 4.46 s


In [437]:
%%time
rec("stories about old civilizations")

CPU times: user 88.9 ms, sys: 5.76 ms, total: 94.6 ms
Wall time: 101 ms


[(3,
  'Feast of Sorrow: A Novel of Ancient Rome',
  'In the twenty-sixth year of Augustus Caesar’s reign, Marcus Gavius Apicius has a singular ambition: to serve as culinary adviser to Caesar. To cement his legacy as Rome’s leading epicure, the wealthy Apicius acquires a young chef, Thrasius, for the exorbitant price of twenty thousand denarii. Apicius believes that the talented Thrasius is the key to his culinary success, and with the slave’s help he soon becomes known for his lavish parties and sumptuous meals. For his part, Thrasius finds a family among Apicius’s household, which includes his daughter, Apicata; his wife, Aelia; and her handmaiden Passia, with whom Thrasius falls passionately in love. But as Apicius draws closer to his ultimate goal, his dangerous single-mindedness threatens his young family and places his entire household at the mercy of the most powerful forces in Rome. “A gastronomical delight” (Associated Press), Feast of Sorrow is a vibrant novel, replete with 